![frames.png](frames.png)

In [1]:
import sympy as sym
import numpy as np
from sympy import sin, cos, Function, Matrix, simplify, lambdify, Eq


In [14]:
# Define helper for generating SE(3) matrix

# rotational
def Rotz(theta):
    return sym.Matrix([
        # 4x4 rotation matrix
        [sym.cos(theta), -sym.sin(theta), 0, 0],
        [sym.sin(theta), sym.cos(theta), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
# translational se(3)
def T(x, y):
    return sym.Matrix([
        # 4x4 translation matrix
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
# Define the helper for hatting a 3x1 vector
def hat(v: sym.Matrix) -> sym.Matrix:
    return sym.Matrix([[0, -v[2], v[1]],
                       [v[2], 0, -v[0]],
                       [-v[1], v[0], 0]])
    
# Define the helper for unhating a 3x3 skew-symmetric matrix
def unhat(v: sym.Matrix) -> sym.Matrix:
    return sym.Matrix([v[2, 1], v[0, 2], v[1, 0]])

# Define the helper for getting the inverse of a g matrix
def inverse_g(g: sym.Matrix) -> sym.Matrix:
    R = g[:3, :3]
    p = g[:3, 3]
    return sym.Matrix([[R.T, -R.T * p], [0, 0, 0, 1]])

def g_dot_SE3(transform, angular_velocity):
    """
    Computes the time derivative of an SE(3) transformation matrix.

    The SE(3) matrix `transform` is composed of a rotation matrix `R` and a position vector `p`.
    The derivative is given as:
        g_dot = [R_dot   p_dot]
                [  0       0 1],

    where:
        - R_dot = R * hat(w),
          `w` is the angular velocity vector.
        - p_dot = derivative of the position vector `p`.

    Parameters
    ----------
    transform : sympy.Matrix
        The SE(3) homogeneous transformation matrix (3x3 in planar case).
    angular_velocity : sympy.Symbol
        Angular velocity (derivative of the rotation angle, theta).

    Returns
    -------
    sympy.Matrix
        Time derivative of the SE(3) homogeneous transformation matrix (g_dot).

    Notes
    -----
    This assumes a 2D planar system with rotation about the z-axis (right-hand rule). The system uses:
        - A 3x3 matrix for rotation and translation.
        - Angular velocity vector w = [0, 0, d(theta)/dt], where z is the axis of rotation.
    """
    # Compute the angular velocity vector (in matrix form for SE(3))
    w = sym.Matrix([0, 0, angular_velocity.diff(t)])
    
    # Extract Components
    R = transform[:3, :3]  # Rotation matrix
    p = transform[:3, 3]   # Translation vector

    # Derivative of rotation: R_dot = R * hat(w)
    R_dot = R * hat(w)

    # Derivative of translation: p_dot
    p_dot = p.diff(t)

    # Construct g_dot
    # use T and Rotz to construct the g_dot matrix
    g_dot_matrix = sym.Matrix([[R_dot, p_dot], [0, 0, 0, 1]])
    return g_dot_matrix

def body_velocity(transform, angular_velocity):
    """
    Computes the body velocity from the SE(3) transformation matrix.

    The body velocity is given as a 6D spatial velocity vector (in planar motion, reduced to 3D):
        V_body = [v  ] -> Linear velocity from R_inv * (p_dot)
                 [w  ] -> Angular velocity

    Parameters
    ----------
    transform : sympy.Matrix
        The SE(3) homogeneous transformation matrix for a frame (3x3 in planar case).
    angular_velocity : sympy.Symbol
        Angular velocity (derivative of the rotation angle, theta).

    Returns
    -------
    sympy.Matrix
        The spatial body velocity vector for the specified frame.

    Raises
    ------
    ValueError
        If an invalid output type is specified.

    Notes
    -----
    - Body velocity is computed as:
          V_body = g^(-1) * g_dot
      where g^(-1) is the inverse of the SE(3) matrix.
    - This function is planar, so angular velocity is scalar (about z-axis).
    """
    # Compute g^-1 (the inverse of SE(3))
    transform_inverse = inverse_g(transform)

    # Compute g_dot
    g_dot_matrix = g_dot_SE3(transform, angular_velocity)

    # Multiply to get body velocity: V_body = g^(-1) * g_dot
    V_body_matrix = transform_inverse * g_dot_matrix

    # Extract body velocity: w = angular component, v = linear component
    w_hat = V_body_matrix[:3, :3]
    v = V_body_matrix[:3, 3]
    w = unhat(w_hat)
    display(v.col_join(w))
    return v.col_join(w)

In [6]:
# We have 6 configuration variables
t = sym.symbols('t')
x_b = sym.Function('x_b')(t)
y_b = sym.Function('y_b')(t)
theta_b = sym.Function('theta_b')(t)
x_j = sym.Function('x_j')(t)
y_j = sym.Function('y_j')(t)
theta_j = sym.Function('theta_j')(t)

In [8]:
# Parameters for the box and jack
l_box, M_box = 6, 100  # Box length and mass
j_box = (1/12) * M_box * (2 * l_box) ** 2  # Moment of inertia for the box

l_jack, m_jack = 1, 1  # Jack length and mass
j_jack = (1/12) * m_jack * (2 * l_jack) ** 2  # Moment of inertia for the jack
M_jack = 1  # Mass of the jack
# Gravitational constant
g = 9.81

# External forces (box shaking parameters)
theta_d_b = sin(2 * np.pi * t / 5)
k = 30000
F_y_b = 4 * M_box * g
F_theta_b = k * theta_d_b
F_ext = Matrix([0, F_y_b, F_theta_b, 0, 0, 0])

# Generalized coordinates
q = Matrix([x_b, y_b, theta_b, x_j, y_j, theta_j])
qdot = q.diff(t)
qddot = qdot.diff(t)

# Homogeneous transformation matrices
g_wa = Rotz(theta_b) * T(x_b, y_b)
g_a_a1 = sym.Matrix([
    [0,  1, 0, -l_box / 2],
    [-1, 0, 0, 0        ],
    [0,  0, 1, 0        ],
    [0,  0, 0, 1        ]
])

g_a_a2 = sym.Matrix([
    [-1,  0, 0, 0],
    [0, -1, 0, l_box / 2],
    [0,  0, 1, 0        ],
    [0,  0, 0, 1        ]
])

g_a_a3 = sym.Matrix([
    [1, 0, 0, 0         ],
    [0, 1, 0, -l_box / 2],
    [0, 0, 1, 0         ],
    [0, 0, 0, 1         ]
])

g_a_a4 = sym.Matrix([
    [0, -1, 0, l_box / 2],
    [1,  0, 0, 0        ],
    [0,  0, 1, 0        ],
    [0,  0, 0, 1        ]
])

g_wb = Rotz(theta_j) * T(x_j, y_j)
g_b_b1 = sym.Matrix([
    [1,  0, 0, -l_jack / 2],
    [0, 1, 0, -l_jack / 2],
    [0,  0, 1, 0         ],
    [0,  0, 0, 1         ]
])

g_b_b2 = sym.Matrix([
    [1,  0, 0, -l_jack / 2],
    [0,  1, 0, l_jack / 2],
    [0,  0, 1, 0         ],
    [0,  0, 0, 1         ]
])

g_b_b3 = sym.Matrix([
    [1,  0, 0, l_jack / 2],
    [0, -1, 0, -l_jack / 2],
    [0,  0, 1, 0         ],
    [0,  0, 0, 1         ]
])

g_b_b4 = sym.Matrix([
    [1,  0, 0, l_jack / 2],
    [0, 1, 0, l_jack / 2],
    [0,  0, 1, 0         ],
    [0,  0, 0, 1         ]
])

In [26]:
origin = sym.Matrix([0, 0, 0, 1])
r_wa = g_wa *origin
r_wb = g_wb *origin

In [30]:
# Now calculate velocities of the box and jack
v_a = body_velocity(g_wa, theta_b)
v_b = body_velocity(g_wb, theta_j)

Matrix([
[-(x_b(t)*sin(theta_b(t)) + y_b(t)*cos(theta_b(t)))*sin(theta_b(t)) - (x_b(t)*cos(theta_b(t)) - y_b(t)*sin(theta_b(t)))*cos(theta_b(t)) + (-x_b(t)*sin(theta_b(t))*Derivative(theta_b(t), t) - y_b(t)*cos(theta_b(t))*Derivative(theta_b(t), t) - sin(theta_b(t))*Derivative(y_b(t), t) + cos(theta_b(t))*Derivative(x_b(t), t))*cos(theta_b(t)) + (x_b(t)*cos(theta_b(t))*Derivative(theta_b(t), t) - y_b(t)*sin(theta_b(t))*Derivative(theta_b(t), t) + sin(theta_b(t))*Derivative(x_b(t), t) + cos(theta_b(t))*Derivative(y_b(t), t))*sin(theta_b(t))],
[-(x_b(t)*sin(theta_b(t)) + y_b(t)*cos(theta_b(t)))*cos(theta_b(t)) + (x_b(t)*cos(theta_b(t)) - y_b(t)*sin(theta_b(t)))*sin(theta_b(t)) - (-x_b(t)*sin(theta_b(t))*Derivative(theta_b(t), t) - y_b(t)*cos(theta_b(t))*Derivative(theta_b(t), t) - sin(theta_b(t))*Derivative(y_b(t), t) + cos(theta_b(t))*Derivative(x_b(t), t))*sin(theta_b(t)) + (x_b(t)*cos(theta_b(t))*Derivative(theta_b(t), t) - y_b(t)*sin(theta_b(t))*Derivative(theta_b(t), t) + sin(theta_

Matrix([
[-(x_j(t)*sin(theta_j(t)) + y_j(t)*cos(theta_j(t)))*sin(theta_j(t)) - (x_j(t)*cos(theta_j(t)) - y_j(t)*sin(theta_j(t)))*cos(theta_j(t)) + (-x_j(t)*sin(theta_j(t))*Derivative(theta_j(t), t) - y_j(t)*cos(theta_j(t))*Derivative(theta_j(t), t) - sin(theta_j(t))*Derivative(y_j(t), t) + cos(theta_j(t))*Derivative(x_j(t), t))*cos(theta_j(t)) + (x_j(t)*cos(theta_j(t))*Derivative(theta_j(t), t) - y_j(t)*sin(theta_j(t))*Derivative(theta_j(t), t) + sin(theta_j(t))*Derivative(x_j(t), t) + cos(theta_j(t))*Derivative(y_j(t), t))*sin(theta_j(t))],
[-(x_j(t)*sin(theta_j(t)) + y_j(t)*cos(theta_j(t)))*cos(theta_j(t)) + (x_j(t)*cos(theta_j(t)) - y_j(t)*sin(theta_j(t)))*sin(theta_j(t)) - (-x_j(t)*sin(theta_j(t))*Derivative(theta_j(t), t) - y_j(t)*cos(theta_j(t))*Derivative(theta_j(t), t) - sin(theta_j(t))*Derivative(y_j(t), t) + cos(theta_j(t))*Derivative(x_j(t), t))*sin(theta_j(t)) + (x_j(t)*cos(theta_j(t))*Derivative(theta_j(t), t) - y_j(t)*sin(theta_j(t))*Derivative(theta_j(t), t) + sin(theta_

In [31]:
# Now calculate inertia
I_a = sym.Matrix([
    [4*M_box, 0, 0, 0, 0, 0],
    [0, 4*M_box, 0, 0, 0, 0],
    [0, 0, 4*M_box, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, j_jack]
])

I_b = sym.Matrix([
    [4*M_jack, 0, 0, 0, 0, 0],
    [0, 4*M_jack, 0, 0, 0, 0],
    [0, 0, 4*M_jack, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, j_jack]
])

In [33]:
# Now calculate the kinetic energy
display(v_a.shape)
display(I_a.shape)
display(v_b.shape)
display(I_b.shape)
KE = sym.simplify((1/2)*v_a.T*I_a*v_a + (1/2)*v_b.T*I_b*v_b)[0]
display(KE)
V = M_box * g * r_wa[1] + M_jack * g * r_wb[1]
display(V)
L = KE - V

# Now calculate the Euler-Lagrange equations
L_qdot = L.diff(qdot)
L_qdot_dot = L_qdot.diff(t)
L_q = L.diff(q)
eqs = sym.simplify(L_q - L_qdot_dot)

lhs = eqs
rhs = F_ext
EL_eqs = Eq(lhs, rhs)
display(EL_eqs)

# Now solve the equations
sol = sym.solve(EL_eqs, qddot)
display(sol)


(6, 1)

(6, 6)

(6, 1)

(6, 6)

200.0*(x_b(t)*Derivative(theta_b(t), t) - y_b(t) + Derivative(y_b(t), t))**2 + 2.0*(x_j(t)*Derivative(theta_j(t), t) - y_j(t) + Derivative(y_j(t), t))**2 + 200.0*(x_b(t) + y_b(t)*Derivative(theta_b(t), t) - Derivative(x_b(t), t))**2 + 2.0*(x_j(t) + y_j(t)*Derivative(theta_j(t), t) - Derivative(x_j(t), t))**2 + 0.166666666666667*Derivative(theta_b(t), t)**2 + 0.166666666666667*Derivative(theta_j(t), t)**2

981.0*x_b(t)*sin(theta_b(t)) + 9.81*x_j(t)*sin(theta_j(t)) + 981.0*y_b(t)*cos(theta_b(t)) + 9.81*y_j(t)*cos(theta_j(t))

Eq(Matrix([
[                                                                                                                                                                                                          400.0*x_b(t)*Derivative(theta_b(t), t)**2 + 400.0*x_b(t) + 400.0*y_b(t)*Derivative(theta_b(t), (t, 2)) - 981.0*sin(theta_b(t)) + 800.0*Derivative(theta_b(t), t)*Derivative(y_b(t), t) - 400.0*Derivative(x_b(t), (t, 2))],
[                                                                                                                                                                                                         -400.0*x_b(t)*Derivative(theta_b(t), (t, 2)) + 400.0*y_b(t)*Derivative(theta_b(t), t)**2 + 400.0*y_b(t) - 981.0*cos(theta_b(t)) - 800.0*Derivative(theta_b(t), t)*Derivative(x_b(t), t) - 400.0*Derivative(y_b(t), (t, 2))],
[-400.0*x_b(t)**2*Derivative(theta_b(t), (t, 2)) - 981.0*x_b(t)*cos(theta_b(t)) - 800.0*x_b(t)*Derivative(theta_b(t), t)*Derivative(x_b(t), t)

{Derivative(theta_b(t), (t, 2)): 11772.0*x_b(t) - 90000.0*sin(1.25663706143592*t),
 Derivative(theta_j(t), (t, 2)): 0.0,
 Derivative(x_b(t), (t, 2)): 11772.0*x_b(t)*y_b(t) + x_b(t)*Derivative(theta_b(t), t)**2 + x_b(t) - 90000.0*y_b(t)*sin(1.25663706143592*t) - 2.4525*sin(theta_b(t)) + 2.0*Derivative(theta_b(t), t)*Derivative(y_b(t), t),
 Derivative(x_j(t), (t, 2)): x_j(t)*Derivative(theta_j(t), t)**2 + x_j(t) - 2.4525*sin(theta_j(t)) + 2.0*Derivative(theta_j(t), t)*Derivative(y_j(t), t),
 Derivative(y_b(t), (t, 2)): -11772.0*x_b(t)**2 + 90000.0*x_b(t)*sin(1.25663706143592*t) + y_b(t)*Derivative(theta_b(t), t)**2 + y_b(t) - 2.4525*cos(theta_b(t)) - 2.0*Derivative(theta_b(t), t)*Derivative(x_b(t), t) - 9.81,
 Derivative(y_j(t), (t, 2)): y_j(t)*Derivative(theta_j(t), t)**2 + y_j(t) - 2.4525*cos(theta_j(t)) - 2.0*Derivative(theta_j(t), t)*Derivative(x_j(t), t)}